In [1]:
import os
import numpy as np

DATA_DIR="Data/"

train_X=np.load(os.path.join(DATA_DIR,"PP_train_X.npy"))
train_y=np.load(os.path.join(DATA_DIR,"PP_train_y.npy"))
test_X=np.load(os.path.join(DATA_DIR,"PP_test_X.npy"))
test_y=np.load(os.path.join(DATA_DIR,"PP_test_X.npy"))

In [2]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
MAX_LEN=69
n_words=4169
n_tags=5
opt="rmsprop"

input = Input(shape=(MAX_LEN,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=MAX_LEN, mask_zero=True)(input)
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model) 
model = TimeDistributed(Dense(50, activation="relu"))(model)
crf = CRF(n_tags)
out = crf(model)

model = Model(input, out)

Using TensorFlow backend.
/home/darp_lord/Installs/anaconda3/envs/NLP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/darp_lord/Installs/anaconda3/envs/NLP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/darp_lord/Installs/anaconda3/envs/NLP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
model.compile(optimizer=opt, loss=crf.loss_function, metrics=[crf.accuracy])
model.summary()

/home/darp_lord/Installs/anaconda3/envs/NLP/lib/python3.7/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/home/darp_lord/Installs/anaconda3/envs/NLP/lib/python3.7/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 69)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 69, 20)            83400     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 69, 100)           28400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 69, 50)            5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 69, 5)             290       
Total params: 117,140
Trainable params: 117,140
Non-trainable params: 0
_________________________________________________________________


In [4]:
from keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
from datetime import datetime

MODEL_NAME="LSTM-CRF"
MODEL_DIR="Models/"+MODEL_NAME

if not os.path.exists(MODEL_DIR):
	os.makedirs(MODEL_DIR)

chkpt_path=os.path.join(MODEL_DIR,opt+"-{epoch:02d}-{val_crf_viterbi_accuracy:.4f}.hdf5")
checkpointer = ModelCheckpoint(chkpt_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

log_path=os.path.join(MODEL_DIR,opt+".log")
csv_logger = CSVLogger(log_path)

tb_path=os.path.join(MODEL_DIR,opt+"_"+datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard=TensorBoard(log_dir=tb_path)

In [6]:
history = model.fit(train_X, 
					train_y, 
					batch_size=32, 
					epochs=10,
                    validation_split=0.1, 
					verbose=1,
					callbacks=[checkpointer,csv_logger,tensorboard])

Instructions for updating:
Use tf.cast instead.
Train on 1060 samples, validate on 118 samples
Epoch 1/10
1060/1060 [==============================] - 8s 8ms/step - loss: 0.3285 - crf_viterbi_accuracy: 0.8947 - val_loss: 0.0727 - val_crf_viterbi_accuracy: 0.9878

Epoch 00001: val_loss improved from inf to 0.07270, saving model to Models/LSTM-CRF/rmsprop-01-0.9878.hdf5
Epoch 2/10
1060/1060 [==============================] - 6s 6ms/step - loss: 0.0955 - crf_viterbi_accuracy: 0.9831 - val_loss: 0.0689 - val_crf_viterbi_accuracy: 0.9878

Epoch 00002: val_loss improved from 0.07270 to 0.06886, saving model to Models/LSTM-CRF/rmsprop-02-0.9878.hdf5
Epoch 3/10
1060/1060 [==============================] - 6s 6ms/step - loss: 0.0878 - crf_viterbi_accuracy: 0.9831 - val_loss: 0.0652 - val_crf_viterbi_accuracy: 0.9878

Epoch 00003: val_loss improved from 0.06886 to 0.06522, saving model to Models/LSTM-CRF/rmsprop-03-0.9878.hdf5
Epoch 4/10
1060/1060 [==============================] - 6s 5ms/step -

In [7]:
%load_ext tensorboard

The tensorboard module is not an IPython extension.
